In [ ]:
!pip install pillow

In [ ]:
import numpy as np
import tensorflow
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD

In [ ]:
image_width = 100
image_height = 100
epochs = 10
num_class = 2
batch_size = 8

In [ ]:
model = keras.applications.inception_v3.InceptionV3(
    include_top=False, weights='imagenet',
    input_tensor=None,
    input_shape= (image_width, image_height, 3),
    pooling=None,
    classes=num_class)

In [ ]:
model.summary()

In [ ]:
for i in range(len(model.layers)-9):
    model.layers[i].trainable = False

In [ ]:
model.summary()

In [ ]:
top_model = Sequential()
top_model.add(model)
top_model.add(Flatten())
top_model.add(Dense(1024))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dense(num_class))
top_model.add(Activation('softmax'))
top_model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
top_model.summary()

In [ ]:
datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=True,
    zca_whitening=False,
    zca_epsilon=1e-6,
    rotation_range=5.,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.0,
    zoom_range=[1-0.2, 1+0.2],
    channel_shift_range=0.4,
    fill_mode='reflect',
    cval=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,)

train_generator = datagen.flow_from_directory(
        'dataset/train',
        target_size=(image_width, image_height),
        batch_size=batch_size,
        classes = ['cats','dogs'],
        class_mode='categorical')

test_generator = datagen.flow_from_directory(
        'dataset/test',
        target_size=(image_width, image_height),
        batch_size=batch_size,
        classes = ['cats','dogs'],
        class_mode='categorical')

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='models/model_inceptionv3_ver1.h5', verbose=2, save_best_only=True)

In [ ]:
top_model.fit_generator(
        train_generator,
        steps_per_epoch=500,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=250,
        callbacks=[checkpointer] )